# import statements

In [263]:
import pandas as pd
import numpy as np

# create dataframe & clean data

In [264]:
df = pd.read_csv("./PythonCW.csv", header=0)
df = df.dropna(axis=1)
df = df.drop(columns=["Date"])
df.head()


,Nvidia,AAPL,BAESY,CJJD,AZN,SAS,TSLA,Saab AB,PFE,GME,HSBC,LVMH
0,0.0281,0.0218,0.0478,0.0909,0.0321,-0.0232,0.0817,0.0439,0.0300,0.0042,0.0231,0.0656
1,-0.0273,-0.0037,0.0151,0.2833,0.0085,0.0040,-0.0787,-0.0957,0.0097,0.0067,0.0273,-0.0023
2,-0.0028,0.0377,0.0339,-0.2727,0.0110,0.0039,0.0811,-0.0256,0.0096,0.0538,0.0048,0.0158
3,0.0394,0.0489,0.0247,0.0000,0.0127,0.0078,0.0665,-0.0189,0.0000,0.1002,0.0235,0.0415
4,-0.0183,-0.0175,-0.0115,-0.0179,-0.0086,0.1089,0.1087,-0.0017,0.0027,-0.0282,-0.0469,-0.0114


# Specify tickers

In [265]:

tickers = {
    1: "Nvidia",
    2: "AAPL",
    3: "BAESY",
    4: "CJJD",
    5: "AZN",
    6: "SAS",
    7: "TSLA",
    8: "Saab AB",
    9: "PFE",
    10: "GME",
    11: "HSBC",
    12: "LVMH"
    }

# Ask user what columns they want

desired_columns = [1, 2, 3]
desired_tickers = [ tickers[i] for i in desired_columns ]
df = df[desired_tickers]

df.head()

# df.selected

,Nvidia,AAPL,BAESY
0,0.0281,0.0218,0.0478
1,-0.0273,-0.0037,0.0151
2,-0.0028,0.0377,0.0339
3,0.0394,0.0489,0.0247
4,-0.0183,-0.0175,-0.0115


# Calculate mean

In [266]:
mean = df.mean().transpose()

mean

Nvidia    0.012052
AAPL      0.005649
BAESY     0.003030
dtype: float64

# Calculate inv covariance

In [267]:
cov = df.cov()
inv_cov = cov.apply(np.reciprocal)
inv_cov

,Nvidia,AAPL,BAESY
Nvidia,283.327784,925.748095,1774.251385
AAPL,925.748095,722.858590,2909.585175
BAESY,1774.251385,2909.585175,794.300774


# Calculate L
#### For each asset mean x cov^-1

e.g nvida & apple
mean(nvid) x cov^-1(appl&nvid) + mean(appl) x cov^-1(appl&nvid)

e.g. nvida & apple & baesy
mean(nvid) x cov^-1(appl&nvid) + mean(appl) x cov^-1(appl&nvid)
mean(bae) x cov^-1(appl&bae) + mean(appl) x cov^-1(appl&bae)
mean(nvid) x cov^-1(appl&nvid) + mean(appl) x cov^-1(appl&nvid)

In [268]:
l_matrix = inv_cov.mul(mean)
print(l_matrix)
l = l_matrix.sum().sum() - sum(np.diag(l_matrix))
l


           Nvidia       AAPL     BAESY
Nvidia   3.414568   5.229591  5.376136
AAPL    11.156794   4.083460  8.816296
BAESY   21.382661  16.436373  2.406800


np.float64(68.3978510588344)

# Calculate M

In [269]:
user_vector = 1
vector = pd.Series([user_vector]*len(desired_columns), index = inv_cov.index)
print(vector)
print(inv_cov)

m_matrix = inv_cov.mul(vector)
print(m_matrix)
m = m_matrix.sum().sum() - sum(np.diag(m_matrix))
m



Nvidia    1
AAPL      1
BAESY     1
dtype: int64
             Nvidia         AAPL        BAESY
Nvidia   283.327784   925.748095  1774.251385
AAPL     925.748095   722.858590  2909.585175
BAESY   1774.251385  2909.585175   794.300774
             Nvidia         AAPL        BAESY
Nvidia   283.327784   925.748095  1774.251385
AAPL     925.748095   722.858590  2909.585175
BAESY   1774.251385  2909.585175   794.300774


np.float64(11219.169309062185)

# Calculate A

In [270]:
a_matrix = inv_cov.mul(vector).mul(mean)
print(a_matrix)
a = a_matrix.sum().sum() - sum(np.diag(a_matrix))
a

           Nvidia       AAPL     BAESY
Nvidia   3.414568   5.229591  5.376136
AAPL    11.156794   4.083460  8.816296
BAESY   21.382661  16.436373  2.406800


np.float64(68.3978510588344)

# Calculate B

In [271]:
b_matrix = inv_cov.mul(mean).transpose().mul(mean).transpose()
print(b_matrix)
b = b_matrix.sum().sum() - sum(np.diag(b_matrix))
b


          Nvidia      AAPL     BAESY
Nvidia  0.041151  0.063025  0.064791
AAPL    0.063025  0.023068  0.049804
BAESY   0.064791  0.049804  0.007293


np.float64(0.35524035074075316)

# Calculate C

In [272]:
c_matrix = inv_cov.mul(vector).transpose().mul(vector).transpose()
print(c_matrix)
c = c_matrix.sum().sum() - sum(np.diag(c_matrix))
c

             Nvidia         AAPL        BAESY
Nvidia   283.327784   925.748095  1774.251385
AAPL     925.748095   722.858590  2909.585175
BAESY   1774.251385  2909.585175   794.300774


np.float64(11219.169309062185)

# Calculate D

In [273]:
d = b * c - (a**2)
d

np.float64(-692.7643890953509)

# Calculate G

In [274]:
g = 1/(d * ( (m*b) - (l*a) ))
g

np.float64(2.0836697643957237e-06)

# Calculate H

In [275]:
h = 1/(d * ( (l*c) - (m*a) ))
h

/var/folders/jy/2kxj96957pd588fj_3z8nxdh0000gn/T/ipykernel_88845/2631217186.py:1: RuntimeWarning: divide by zero encountered in scalar divide
  h = 1/(d * ( (l*c) - (m*a) ))


np.float64(-inf)